In [13]:
import os
from moviepy.editor import VideoFileClip, concatenate_videoclips, TextClip, ImageClip, CompositeVideoClip, AudioFileClip
import pyttsx3 as tts
import whisper

def speak(text):
    engine = tts.init()
    engine.save_to_file(text, 'samp.mp3')
    engine.runAndWait()
    

def get_files_in_directory(directory):
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]


In [ ]:
def speech_to_text(audio_file):
    model = whisper.load_model("base")
    result = model.transcribe(audio = audio_file)

    start = [result['segments'][i]['start'] for i in range(len(result['segments']))]
    end = [result['segments'][j]['end'] for j in range(len(result['segments']))]
    text = [result['segments'][k]['text'] for k in range(len(result['segments']))]

    return [start, end, text]

speech = speech_to_text("samp.mp3")
speech

In [ ]:
for l in range(len(speech[0])):
    print(((speech[0][l], speech[1][l]), speech[2][l]))

In [ ]:
from moviepy.video.tools.subtitles import SubtitlesClip

def generate_video():
    with open('samp.txt', 'r') as file:
        text = file.read()
    
    lst = text.split(' ')

    clips = []

    generator = lambda txt: TextClip(txt, 
                                fontsize=70, 
                                color='white', 
                                bg_color='none', 
                                font='Arial-Bold',
                                method = 'caption',
                                size = (1920, 1080))

    clips = [CompositeVideoClip([ImageClip('Z_1.jpg').set_duration(0.5).set_position('center')], size=(1920, 1080)) for _ in lst]

    subs = [((speech[0][k], speech[1][k]), speech[2][k]) for k in range(len(speech[0]))]
        

    subtitles = SubtitlesClip(subs, generator)

    final = concatenate_videoclips(clips)
    audio = AudioFileClip("samp.mp3")
    final = final.set_audio(audio)
    final_complete = CompositeVideoClip([final, subtitles.set_position(('center', 'bottom'))])
    final_complete.write_videofile("output0.mp4", fps=24, codec = 'libx264')

In [ ]:
generate_video()

NameError: name 'speech' is not defined